In [1]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

In [71]:
from data.bioasq_dataset import BioASQDataset
from utilities.file_utils import Utils as utils


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
dataset = BioASQDataset('test', tokenizer, True) 

Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-5.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-3.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-4.json


...


Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-2.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-5.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-6b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-5b-1.json



Loading and preprocessing file: /Users/ordonez2/Desktop/cs221/final project/domain_adaptation_QA/data/BioASQ/BioASQ-test-factoid-4b-1.json


In [73]:
encoded_inputs, idx, ans_idx, answer_text = dataset[0]

In [76]:
ans_idx

-1

In [39]:
out[-1]


[{'text': 'Bazex syndrome', 'answer_start': 31}]

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')


In [3]:
context = "The US has passed the peak on new coronavirus cases, " \
          "President Donald Trump said and predicted that some states would reopen this month. " \
          "The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world."

question = "What was President Donald Trump's prediction?"

encoding = tokenizer.encode_plus(question, context)


In [4]:
encoding

{'input_ids': [101, 2054, 2001, 2343, 6221, 8398, 1005, 1055, 17547, 1029, 102, 1996, 2149, 2038, 2979, 1996, 4672, 2006, 2047, 21887, 23350, 3572, 1010, 2343, 6221, 8398, 2056, 1998, 10173, 2008, 2070, 2163, 2052, 2128, 26915, 2023, 3204, 1012, 1996, 2149, 2038, 2058, 6191, 2581, 1010, 2199, 4484, 2522, 17258, 1011, 2539, 3572, 1998, 2058, 2382, 1010, 6445, 2575, 6677, 1010, 1996, 3284, 2005, 2151, 2406, 1999, 1996, 2088, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]


In [18]:
out = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))
start_scores, end_scores = out[0], out[1]

In [20]:
ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)

In [21]:
ans_tokens

[2070, 2163, 2052, 2128, 26915, 2023, 3204]

In [22]:
answer_tokens

['some', 'states', 'would', 're', '##open', 'this', 'month']

In [23]:
print ("\nQuestion ",question)
print ("\nAnswer Tokens: ")
print (answer_tokens)


Question  What was President Donald Trump's prediction?

Answer Tokens: 
['some', 'states', 'would', 're', '##open', 'this', 'month']


In [24]:
answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)


In [25]:
answer_tokens_to_string

'some states would reopen this month'

## Batch Inference 

In [41]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

context = "The US has passed the peak on new coronavirus cases, " \
          "President Donald Trump said and predicted that some states would reopen this month." \
          "The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, " \
          "the highest for any country in the world."

print ("\n\nContext : ",context)

questions = ["What was President Donald Trump's prediction?",
             "How many deaths have been reported from the virus?",
             "How many cases have been reported in the United States?"]

question_context_for_batch = []

for question in questions :
    question_context_for_batch.append((question, context))

encoding = tokenizer.batch_encode_plus(question_context_for_batch,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
out = model(input_ids, attention_mask=attention_mask)
start_scores, end_scores = out[0], out[1]


for index,(start_score,end_score,input_id) in enumerate(zip(start_scores,end_scores,input_ids)):
    max_startscore = torch.argmax(start_score)
    max_endscore = torch.argmax(end_score)
    ans_tokens = input_ids[index][max_startscore: max_endscore + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print ("\nQuestion: ",questions[index])
    print ("Answer: ", answer_tokens_to_string)



Context :  The US has passed the peak on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.

Question:  What was President Donald Trump's prediction?
Answer:  some states would reopen this month

Question:  How many deaths have been reported from the virus?
Answer:  30 , 826

Question:  How many cases have been reported in the United States?
Answer:  over 637 , 000


In [44]:
start_scores.size()

torch.Size([3, 72])

In [51]:
torch.argmax(start_scores, dim = 1)

tensor([30, 55, 43])